# Circuit VS_RL1

In [ ]:
import villas.dataprocessing.readtools as rt
import villas.dataprocessing.plottools as pt
from villas.dataprocessing.timeseries import TimeSeries as ts
import dpsimpy

## Powerflow for Initialization

In [ ]:
sim_name_pf = 'VS_RL1_PF'
dpsimpy.Logger.set_log_dir("logs/" + sim_name_pf)

# Nodes
gnd_pf = dpsimpy.sp.SimNode.gnd
n1_pf  = dpsimpy.sp.SimNode('n1_pf')
n2_pf  = dpsimpy.sp.SimNode('n2_pf')

# Components
vs_pf = dpsimpy.sp.ph1.VoltageSource('vs')
vs_pf.set_parameters(V_ref=complex(10,0), f_src=50)
r1_pf = dpsimpy.sp.ph1.Resistor('r1_pf')
r1_pf.set_parameters(R=5)
l1_pf = dpsimpy.sp.ph1.Inductor('l1_pf')
l1_pf.set_parameters(L=0.02)

# Connections
vs_pf.connect([gnd_pf, n1_pf])
r1_pf.connect([n2_pf, n1_pf])
l1_pf.connect([gnd_pf, n2_pf])

# Define system topology
system_pf = dpsimpy.SystemTopology(50, [n1_pf, n2_pf], [vs_pf, r1_pf, l1_pf])

# Logging
logger_pf = dpsimpy.Logger(sim_name_pf)
logger_pf.log_attribute('n1.v', 'v', n1_pf)
logger_pf.log_attribute('n2.v', 'v', n2_pf)

sim_pf = dpsimpy.Simulation(sim_name_pf, dpsimpy.LogLevel.debug)
sim_pf.set_system(system_pf)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_time_step(0.1)
sim_pf.set_final_time(0.5)
sim_pf.add_logger(logger_pf)
sim_pf.run()

## DPsim EMT

In [ ]:
name = 'EMT_VS_RL1'
dpsimpy.Logger.set_log_dir("logs/" + name)

# Nodes
gnd = dpsimpy.emt.SimNode.gnd
n1 =  dpsimpy.emt.SimNode('n1')
n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0]))
n2 =  dpsimpy.emt.SimNode('n2')
n2.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0]))

# Components
vs = dpsimpy.emt.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0), f_src=50)
r1 = dpsimpy.emt.ph1.Resistor('r1')
r1.set_parameters(R=5)
l1 = dpsimpy.emt.ph1.Inductor('l1')
l1.set_parameters(L=0.02)

vs.connect([gnd, n1])
r1.connect([n2, n1])
l1.connect([gnd, n2])

system = dpsimpy.SystemTopology(50, [gnd, n1, n2], [vs, l1, r1])

logger = dpsimpy.Logger(name)
logger.log_attribute('n1.v', 'v', n1)
logger.log_attribute('n2.v', 'v', n2)
logger.log_attribute('r1.v_intf', 'v_intf', r1)
logger.log_attribute('r1.i_intf', 'i_intf', r1)
logger.log_attribute('l1.v_intf', 'v_intf', l1)
logger.log_attribute('l1.i_intf', 'i_intf', l1)

sim = dpsimpy.Simulation(name)
sim.add_logger(logger)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.EMT)
sim.set_time_step(0.0001)
sim.set_final_time(0.1)

sim.run()

In [ ]:
# read EMT results
#work_dir = '../../dpsim/Logs/'
work_dir = 'logs/EMT_VS_RL1/'
log_name = 'EMT_VS_RL1'
print(work_dir + log_name + '.csv')

ts_dpsim_emt = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
v1 = 'n1.v'
v2 = 'n2.v'
i12 = 'r1.i_intf'

ts_dpsim_emt['n1.v'].label = 'v1 EMT'
ts_dpsim_emt['n2.v'].label = 'v2 EMT'
ts_dpsim_emt['r1.i_intf'].label = 'i12 EMT'
pt.plot_timeseries(1, ts_dpsim_emt['n1.v'])
pt.plot_timeseries(1, ts_dpsim_emt['n2.v'])
pt.plot_timeseries(2, ts_dpsim_emt['r1.i_intf'])

## DPsim DP

In [ ]:
name = 'DP_VS_RL1'
dpsimpy.Logger.set_log_dir("logs/" + name)

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 =  dpsimpy.dp.SimNode('n1')
n1.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n1_pf.name(), 'v').get()[0][0]))
n2 =  dpsimpy.dp.SimNode('n2')
n2.set_initial_voltage(dpsimpy.Math.single_phase_variable_to_three_phase(sim_pf.get_idobj_attr(n2_pf.name(), 'v').get()[0][0]))

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10,0))
r1 = dpsimpy.dp.ph1.Resistor('r1')
r1.set_parameters(R=5)
l1 = dpsimpy.dp.ph1.Inductor('l1')
l1.set_parameters(L=0.02)

vs.connect([gnd, n1])
r1.connect([n2, n1])
l1.connect([gnd, n2])

system = dpsimpy.SystemTopology(50, [gnd, n1, n2], [vs, l1, r1])

logger = dpsimpy.Logger(name)
logger.log_attribute('n1.v', 'v', n1);
logger.log_attribute('n2.v', 'v', n2);
logger.log_attribute('r1.i_intf', 'i_intf', r1);

sim = dpsimpy.Simulation(name)
sim.add_logger(logger)
sim.set_system(system)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_time_step(0.0001)
sim.set_final_time(0.1)

sim.run()

In [ ]:
# read DPsim DP results
work_dir = 'logs/DP_VS_RL1/'
log_name = 'DP_VS_RL1'
print(work_dir + log_name + '.csv')

ts_dpsim_dp = rt.read_timeseries_dpsim(work_dir + log_name + '.csv')

In [ ]:
# convert to emt
ts_dpsim_dp_emt = ts.frequency_shift_list(ts_dpsim_dp, 50)

In [ ]:
pt.set_timeseries_labels(ts_dpsim_dp_emt['n1.v_shift'], 'v1 DP')
pt.set_timeseries_labels(ts_dpsim_dp_emt['n2.v_shift'], 'v2 DP')
pt.set_timeseries_labels(ts_dpsim_dp_emt['r1.i_intf_shift'], 'i12 DP')
pt.plot_timeseries(1, ts_dpsim_dp_emt['n1.v_shift'])
pt.plot_timeseries(1, ts_dpsim_dp_emt['n2.v_shift'])
pt.plot_timeseries(2, ts_dpsim_dp_emt['r1.i_intf_shift'])

## Modelica reference results

In [ ]:
# read Simulink log file
import os
import urllib.request

if not os.path.exists('reference-results'):
    os.mkdir('reference-results')

url = 'https://raw.githubusercontent.com/dpsim-simulator/reference-results/master/Modelica/BasicGrids/VS_RL1.csv'
local_file = 'reference-results/MOD_VS_RL1.csv'
urllib.request.urlretrieve(url, local_file) 

ts_mod = rt.read_timeseries_dpsim(local_file)

In [ ]:
pt.set_timeseries_labels(ts_mod['voltageSource.v'], 'v1 Modelica')
pt.set_timeseries_labels(ts_mod['inductor.v'], 'v2 Modelica')
pt.set_timeseries_labels(ts_mod['inductor.i'], 'i12 Modelica')
pt.plot_timeseries(1, ts_mod['voltageSource.v'])
pt.plot_timeseries(1, ts_mod['inductor.v'])
pt.plot_timeseries(2, ts_mod['inductor.i'])

## Comparison

In [ ]:
v1 = 'n1.v'
v2 = 'n2.v'
i12 = 'r1.i_intf'

# plot v1
pt.plot_timeseries(1, ts_dpsim_emt['n1.v'])
pt.plot_timeseries(1, ts_dpsim_dp_emt['n1.v_shift'])
pt.plot_timeseries(1, ts_mod['voltageSource.v'])
# plot v2
pt.plot_timeseries(2, ts_dpsim_emt['n2.v'])
pt.plot_timeseries(2, ts_dpsim_dp_emt['n2.v_shift'])
pt.plot_timeseries(2, ts_mod['inductor.v'])
# plot i12
pt.plot_timeseries(3, ts_dpsim_emt['r1.i_intf'])
pt.plot_timeseries(3, ts_dpsim_dp_emt['r1.i_intf_shift'])
pt.plot_timeseries(3, ts_mod['inductor.i'])

In [ ]:
# calculate the RMSE between modelica (ts_mod) and EMT (ts_dpsim_emt)
err_mod_emt = 0
err_mod_emt += ts.rmse(ts_mod['voltageSource.v'], ts_dpsim_emt['n1.v'])
err_mod_emt += ts.rmse(ts_mod['inductor.v'], ts_dpsim_emt['n2.v'])
err_mod_emt += ts.rmse(ts_mod['inductor.i'], ts_dpsim_emt['r1.i_intf'])
err_mod_emt = err_mod_emt / 3
print("Total RMSE of Simulink reference and DPsim EMT: %g" % (err_mod_emt))

# calculate the RMSE between Simulink (ts_sl) and DP (ts_dpsim_dp_emt)
err_mod_dp = 0
err_mod_dp += ts.rmse(ts_mod['voltageSource.v'], ts_dpsim_dp_emt['n1.v_shift'])
err_mod_dp += ts.rmse(ts_mod['inductor.v'], ts_dpsim_dp_emt['n2.v_shift'])
err_mod_dp += ts.rmse(ts_mod['inductor.i'],  ts_dpsim_dp_emt['r1.i_intf_shift'])
err_mod_dp = err_mod_dp / 3
print("Total RMSE of Simulink reference and DPsim DP: %g" % (err_mod_dp))

In [ ]:
assert err_mod_emt < 0.00012
assert err_mod_dp < 0.0000041